In [1]:
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
FLAGS = tf.app.flags.FLAGS

定义batch_size 迭代次数 和cifar-10的数据默认路径

In [3]:
max_steps = 3000
batch_size = 128
data_dir = '/home/hhl/machine_learning_tutorial/tensorflow_tutorial/advance_cnn/data'

 * 准备数据

maybe_download_and_extract() 报错
UnrecognizedFlagError: Unknown command line flag 'f'

解决方案 tf.app.flags.DEFINE_string('f', '', 'kernel')

In [4]:
# tf.app.flags.DEFINE_string('f', '', 'kernel')

In [5]:
# cifar10.maybe_download_and_extract()

In [6]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir = data_dir,batch_size = batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [7]:
images_test,labels_test = cifar10_input.inputs(eval_data = True, data_dir = data_dir, batch_size = batch_size)

* var 是正常的参数，l2_loss 是将var的正则未来要放到loss中，w1是该正则的系数。

In [8]:
def variable_with_weigth_loss(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev = stddev))
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [9]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24,24, 3])
label_holder = tf.placeholder(tf.float32, [batch_size])

 * 第一层卷积

In [10]:
weight1 = variable_with_weigth_loss(shape = [5, 5, 3, 64], stddev = 0.05, w1 = 0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding = 'SAME')
bias1 = tf.Variable(tf.constant(0.0, shape = [64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
pool1 = tf.nn.max_pool(conv1, ksize = [1, 3, 3, 1], strides = [1, 2, 2, 1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias = 1.0, alpha = 0.001 / 9.0, beta = 0.75)

* 第二层卷积

In [11]:
weight2 = variable_with_weigth_loss(shape = [5, 5, 64, 64], stddev = 0.05, w1 = 0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding = 'SAME')
bias2 = tf.Variable(tf.constant(0.1, shape = [64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias = 1.0, alpha=0.001/9.9,beta = 0.75)
pool2 = tf.nn.max_pool(norm2, ksize = [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

* 全链接层

In [12]:
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weigth_loss(shape=[dim,384], stddev=0.04, w1=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

* 第二全链接层

In [13]:
weight4 = variable_with_weigth_loss(shape = [384,192], stddev = 0.04, w1 = 0.004)
bias4 = tf.Variable(tf.constant(0.1, shape = [192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

* 输出

In [14]:
weight5 = variable_with_weigth_loss(shape = [192, 10], stddev=1/192.0, w1 = 0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5),bias5)

 * 以上即使inference 的部分，即正向传播的部分。
 
 下面开始计算loss

In [15]:
def loss(logits,labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, 
                                                                   labels = labels, 
                                                                   name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name = 'cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')

In [16]:
loss = loss(logits,label_holder)

In [17]:
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [19]:
label_holder2 = tf.placeholder(tf.int32, [batch_size])

In [20]:
top_k_op = tf.nn.in_top_k(logits,label_holder2,1)

In [21]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [22]:
tf.train.start_queue_runners()

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 139887193868032)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139887185475328)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139887177082624)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139887168689920)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139886682175232)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139886673782528)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139886665389824)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [23]:
for step in (range(max_steps)):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict = {image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time
    if step % 10 == 0:
        example_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        format_str = ('step %d, loss = %.2f ( %.1f example/sec; %.3f sec/batch)')
        print(format_str % (step,loss_value, example_per_sec,sec_per_batch))

step 0, loss = 4.68 ( 12.5 example/sec; 10.206 sec/batch)
step 10, loss = 3.68 ( 1760.4 example/sec; 0.073 sec/batch)
step 20, loss = 3.05 ( 1791.1 example/sec; 0.071 sec/batch)
step 30, loss = 2.70 ( 1949.7 example/sec; 0.066 sec/batch)
step 40, loss = 2.38 ( 1866.6 example/sec; 0.069 sec/batch)
step 50, loss = 2.35 ( 1670.9 example/sec; 0.077 sec/batch)
step 60, loss = 2.12 ( 2090.5 example/sec; 0.061 sec/batch)
step 70, loss = 2.17 ( 1826.4 example/sec; 0.070 sec/batch)
step 80, loss = 1.92 ( 1507.4 example/sec; 0.085 sec/batch)
step 90, loss = 1.95 ( 1598.1 example/sec; 0.080 sec/batch)
step 100, loss = 1.97 ( 2181.3 example/sec; 0.059 sec/batch)
step 110, loss = 1.90 ( 1668.2 example/sec; 0.077 sec/batch)
step 120, loss = 1.95 ( 1815.5 example/sec; 0.071 sec/batch)
step 130, loss = 1.84 ( 1838.5 example/sec; 0.070 sec/batch)
step 140, loss = 1.83 ( 1763.5 example/sec; 0.073 sec/batch)
step 150, loss = 1.94 ( 1770.1 example/sec; 0.072 sec/batch)
step 160, loss = 1.79 ( 1664.6 examp

step 1340, loss = 1.28 ( 1725.6 example/sec; 0.074 sec/batch)
step 1350, loss = 1.29 ( 1695.1 example/sec; 0.076 sec/batch)
step 1360, loss = 1.22 ( 1784.6 example/sec; 0.072 sec/batch)
step 1370, loss = 1.41 ( 1808.1 example/sec; 0.071 sec/batch)
step 1380, loss = 1.17 ( 1915.3 example/sec; 0.067 sec/batch)
step 1390, loss = 1.20 ( 1755.8 example/sec; 0.073 sec/batch)
step 1400, loss = 1.28 ( 2043.6 example/sec; 0.063 sec/batch)
step 1410, loss = 1.11 ( 1811.4 example/sec; 0.071 sec/batch)
step 1420, loss = 1.02 ( 1704.6 example/sec; 0.075 sec/batch)
step 1430, loss = 1.04 ( 1871.2 example/sec; 0.068 sec/batch)
step 1440, loss = 1.14 ( 1693.1 example/sec; 0.076 sec/batch)
step 1450, loss = 1.19 ( 1932.2 example/sec; 0.066 sec/batch)
step 1460, loss = 1.14 ( 1635.7 example/sec; 0.078 sec/batch)
step 1470, loss = 1.08 ( 1604.2 example/sec; 0.080 sec/batch)
step 1480, loss = 1.38 ( 1858.8 example/sec; 0.069 sec/batch)
step 1490, loss = 1.21 ( 1925.8 example/sec; 0.066 sec/batch)
step 150

step 2670, loss = 0.88 ( 1690.0 example/sec; 0.076 sec/batch)
step 2680, loss = 0.95 ( 1941.6 example/sec; 0.066 sec/batch)
step 2690, loss = 1.19 ( 1830.7 example/sec; 0.070 sec/batch)
step 2700, loss = 1.02 ( 1757.7 example/sec; 0.073 sec/batch)
step 2710, loss = 1.01 ( 1867.0 example/sec; 0.069 sec/batch)
step 2720, loss = 1.09 ( 1858.6 example/sec; 0.069 sec/batch)
step 2730, loss = 0.98 ( 1838.2 example/sec; 0.070 sec/batch)
step 2740, loss = 0.97 ( 1940.0 example/sec; 0.066 sec/batch)
step 2750, loss = 1.14 ( 1724.8 example/sec; 0.074 sec/batch)
step 2760, loss = 1.12 ( 1965.1 example/sec; 0.065 sec/batch)
step 2770, loss = 1.19 ( 1436.1 example/sec; 0.089 sec/batch)
step 2780, loss = 1.03 ( 1803.6 example/sec; 0.071 sec/batch)
step 2790, loss = 1.09 ( 1663.2 example/sec; 0.077 sec/batch)
step 2800, loss = 1.17 ( 1786.5 example/sec; 0.072 sec/batch)
step 2810, loss = 1.10 ( 1537.7 example/sec; 0.083 sec/batch)
step 2820, loss = 1.25 ( 1740.6 example/sec; 0.074 sec/batch)
step 283

In [24]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch , label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op],feed_dict = {image_holder:image_batch,label_holder2:label_batch})
#     test_acc = accuracy.eval(feed_dict = {image_holder:image_batch,label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1

In [26]:
precision = true_count / total_sample_count

In [27]:
precision

0.7093552215189873